In [1]:
from PIL import Image
from glob import glob
import cv2
from cv2 import aruco
import numpy as np
from sympy import *
import math


In [2]:
path = '/mnt/hdd/Bilder/Bilder/2022/2022-10-23_Heilgarten/'


In [5]:
bilder = glob(path + '*.JPG')
for bild in bilder:
    img = Image.open(bild)
    exif = img.getexif()
    print(exif)


{296: 2, 282: 72.0, 34853: 686, 34665: 182, 270: 'DCIM\\101MEDIA\\DJI_0097.JPG', 271: 'DJI\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 272: 'FC7303\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 305: 'v01.43.0067\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 274: 1, 306: '2022:10:23 12:47:11', 531: 1, 283: 72.0, 40092: b'T\x00y\x00p\x00e\x00=\x00N\x00,\x00 \x00M\x00o\x00d\x00e\x00=\x00P\x00,\x00 \x00D\x00E\x00=\x00N\x00o\x00n\x00e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Marker identifizieren

In [7]:

import json


LUT_IN = [0, 158, 216, 255]
LUT_OUT = [0, 22, 80, 176]
lut = np.interp(np.arange(0, 256),
                LUT_IN, LUT_OUT).astype(np.uint8)

aruco_dict = aruco.Dictionary_create(32, 3)
beobachtungen = []

def add_beobachtung(punkt, bild, x, y):
    for punktbeo in beobachtungen:
        if punktbeo['id'] == str(punkt):
            punktbeo["observations"].append({
                "projection": [x,y],
                "shot_id": bild
            })
            return
    beobachtungen.append({
        "id": str(punkt),
        "observations": [
            {
                "projection": [x,y],
                "shot_id": bild
            }
        ]
    })


for bildnr in range(len(bilder)):
    bild = bilder[bildnr]
    cv_img = cv2.imread(bild)
    height, width, channels = cv_img.shape
    maxP = max([height, width])/2.
    tmp = cv2.LUT(cv_img, lut)
    gray = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
    
    parameter = aruco.DetectorParameters.create()
    parameter.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX;
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=parameter)
    if ids is None: continue;
    for nr in range(len(ids)):
        for cid in range(len(corners[nr][0])):                
            #image, track_id, feature_id, x, y, r, g, b
            x = (corners[nr][0][cid][0]-width/2.)/maxP
            y = (corners[nr][0][cid][1]-height/2.)/maxP
            id = ids[nr][0]*10 + cid
            add_beobachtung(id, bild, x, y)
beobachtungen
js = json.dumps({"points":beobachtungen})
file = open(path + 'ground_control_points.json','w')
file.write(js)
file.close()
